In [ ]:
import csv
import re

month_map = {
    'january': '01', 'jan': '01',
    'february': '02', 'feb': '02',
    'march': '03', 'mar': '03',
    'april': '04', 'apr': '04',
    'may': '05',
    'june': '06', 'jun': '06',
    'july': '07', 'jul': '07',
    'august': '08', 'aug': '08',
    'september': '09', 'sep': '09', 'sept': '09',
    'october': '10', 'oct': '10',
    'november': '11', 'nov': '11',
    'december': '12', 'dec': '12'
}

def parse_date(text):
    text = text.lower()
    # Remove ordinal suffixes like 1st, 2nd, 3rd, 4th
    text = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', text)

    # Pattern 1: yyyy.mm.dd (e.g., 2023.07.25)
    # Matches 4-digit year, dot, 1-2 digit month, dot, 1-2 digit day
    match = re.search(r'(\d{4})[.](\d{1,2})[.](\d{1,2})', text)
    if match:
        yyyy, mm, dd = match.groups()
        return f"{dd.zfill(2)}-{mm.zfill(2)}-{yyyy}"

    # Pattern 2: yyyy-mm-dd or yyyy/mm/dd (e.g., 2023-07-25 or 2023/07/25)
    # Matches 4-digit year, dash or slash, 1-2 digit month, dash or slash, 1-2 digit day
    match = re.search(r'(\d{4})[-/](\d{1,2})[-/](\d{1,2})', text)
    if match:
        yyyy, mm, dd = match.groups()
        return f"{dd.zfill(2)}-{mm.zfill(2)}-{yyyy}"

    # Pattern 3: mm/dd/yyyy or dd/mm/yyyy (e.g., 07/25/2023 or 25/07/2023)
    # Ambiguous US vs non-US format; uses heuristic: if first number > 12, assume dd/mm/yyyy
    match = re.search(r'(\d{1,2})/(\d{1,2})/(\d{4})', text)
    if match:
        mm, dd, yyyy = match.groups()
        if int(mm) > 12:  # Treat as dd/mm/yyyy if first number > 12
            dd, mm = mm, dd
        return f"{dd.zfill(2)}-{mm.zfill(2)}-{yyyy}"

    # Pattern 4: dd.mm.yy or dd/mm/yy or dd-mm-yyyy (e.g., 25.07.23 or 25/07/2023)
    # Matches 1-2 digit day, separator, 1-2 digit month, separator, 2 or 4 digit year
    match = re.search(r'(\d{1,2})[./-](\d{1,2})[./-](\d{2,4})', text)
    if match:
        dd, mm, yy = match.groups()
        # Convert 2-digit year to 4-digit based on cutoff
        if len(yy) == 2:
            yy = '20' + yy if int(yy) < 30 else '19' + yy
        return f"{dd.zfill(2)}-{mm.zfill(2)}-{yy}"

    # Pattern 5: "5 of March 2023" or "5 March 2023"
    # Matches 1-2 digit day, optional "of", month name, optional comma, 4-digit year
    match = re.search(r'(\d{1,2})\s*(?:of)?\s*([a-z]+)[,]?\s*(\d{4})', text)
    if match:
        dd, month, yyyy = match.groups()
        mm = month_map.get(month.lower())
        if mm:
            return f"{dd.zfill(2)}-{mm}-{yyyy}"

    # Pattern 6: "March 5, 2023"
    # Matches month name, 1-2 digit day, optional comma, 4-digit year
    match = re.search(r'([a-z]+)\s+(\d{1,2}),?\s*(\d{4})', text)
    if match:
        month, dd, yyyy = match.groups()
        mm = month_map.get(month.lower())
        if mm:
            return f"{dd.zfill(2)}-{mm}-{yyyy}"

    # Pattern 7: "the 4 of July, 2022"
    # Matches "the", 1-2 digit day, optional "of", month name, optional comma, 4-digit year
    match = re.search(r'the\s+(\d{1,2})\s+(?:of\s+)?([a-z]+),?\s*(\d{4})', text)
    if match:
        dd, month, yyyy = match.groups()
        mm = month_map.get(month.lower())
        if mm:
            return f"{dd.zfill(2)}-{mm}-{yyyy}"

    # Pattern 8: Fallback - Look for day and month (e.g., "25th December") and year elsewhere
    # Extracts any 1-2 digit day + month name combination, and any 4-digit year separately
    dm_match = re.search(r'(\d{1,2})\s*(?:of\s+)?([a-z]{3,9})', text)
    y_match = re.search(r'(\d{4})', text)
    if dm_match and y_match:
        dd, month = dm_match.groups()
        mm = month_map.get(month.lower())
        yyyy = y_match.group(1)
        if mm:
            return f"{dd.zfill(2)}-{mm}-{yyyy}"

    # No recognizable date format found
    return "No date found"



# Evaluation function
def evaluate_on_csv(file_path):
    total, correct = 0, 0
    mismatches = []

    with open(file_path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            input_text = row['Input']
            expected = row['Expected Output']
            predicted = parse_date(input_text)

            if predicted == expected:
                correct += 1
            else:
                mismatches.append((input_text, predicted, expected))
            total += 1

    accuracy = correct / total * 100 if total else 0
    print(f"\n✅ Accuracy: {accuracy:.2f}% ({correct}/{total})\n")

    if mismatches:
        print("❌ Mismatches:")
        for input_text, predicted, expected in mismatches:
            print(f"Input: {input_text}")
            print(f"Predicted: {predicted}")
            print(f"Expected : {expected}")
            print("---")

# Run on your CSV file
evaluate_on_csv("date_parser_testcases.csv")




✅ Accuracy: 95.96% (95/99)

❌ Mismatches:
Input: Her birthday is on 07/08/1990.
Predicted: 08-07-1990
Expected : 07-08-1990
---
Input: The interview is on 1/2/2022.
Predicted: 02-01-2022
Expected : 01-02-2022
---
Input: Her birthday, which she celebrates on 07/08/1990, is coming up soon.
Predicted: 08-07-1990
Expected : 07-08-1990
---
Input: The job interview is on 1/2/2022, don't be late.
Predicted: 02-01-2022
Expected : 01-02-2022
---
